# Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
KEYWORDS = ['python', 'парсинг','SSD','Марс'] 

In [2]:
res = requests.get('https://habr.com/ru/all/')
soup_ = BeautifulSoup(res.text, 'lxml')


In [39]:
#собираю список ссылок со страницы
news = soup_.find_all('a', class_ = "post__title_link")

href_list = []
for new in news:
    href_ = new.get('href')
    href_list.append(href_)
    
#href_list


In [40]:
# Функция проверки текста на наличие ключей
def check_paper (KEYWORDS,text):
    for key in KEYWORDS:
        if key in str(text):        
            return True
      
#check_paper (KEYWORDS,text) 

In [48]:
#Создаю и наполняю датафрейм 
frame = pd.DataFrame(columns=['Заголовок', 'Дата', 'Ссылка', 'Текст статьи'])

for link in href_list:
    soup_ = BeautifulSoup(requests.get(link).text, 'lxml')
    text = soup_.find('div', class_ = 'post__body post__body_full').text
    check = check_paper (KEYWORDS,text) 
    if check == True:
        heder = soup_.title.text
        date = soup_.find('span', class_ = 'post__time').get('data-time_published')
        df2 = pd.DataFrame({'Заголовок' : [heder], 'Дата' : [date], 'Ссылка' : [link], 'Текст статьи' : [text]})
        frame = pd.concat([frame, df2])
   
    time.sleep(0.3)

frame  

,Заголовок,Дата,Ссылка,Текст статьи
0,Учёные раскрыли универсальную геометрию геолог...,2020-12-18T10:35Z,https://habr.com/ru/post/533752/,\nУпражнения в чистой математике привели к соз...
0,Энергия и энтропия / Хабр,2020-12-18T10:18Z,https://habr.com/ru/post/533740/,\nЭта статья является конспектом материала пос...
0,Марс — наш / Блог компании VDSina.ru хостинг с...,2020-12-18T09:45Z,https://habr.com/ru/company/vdsina/blog/532430/,\n\n\r\nМарс — потенциально пригодная для жизн...


# Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [49]:
EMAIL = ['xx@x.ru', 'yyy@y.com']

In [5]:
#Тут так и не понял, где искать post запрос.

URL = 'https://www.avast.com/hackcheck'
data = {}
req = requests.post(URL, data)
req


<Response [405]>

Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [21]:
f = open ('/Users/maksim/Netology/material/token.txt')
token = f.read()
print(token)

b94f71df2d135dced40605fe20535d066d711c9121c75ec76cae2f97db53608148c99f296029ca98b6b42


In [24]:
wall_get = 'https://api.vk.com/method/wall.get?'
version = '5.84'
GROUP = 'netology' 
SLEEP = 0.33

In [25]:
params =  {
    'access_token': token,
    'v': version,
    'count': 10,
    'domain': GROUP
}

In [34]:
res = requests.get(wall_get, params)
#res.json()

In [61]:
from datetime import datetime
data = pd.DataFrame(res.json()['response']['items'], columns=['date', 'text'])
data
#ts = data['date'][1]
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M'))

,date,text
0,1607940301,Наш предновогодний digital-марафон продолжаетс...
1,1608450780,"Предсказать будущее невозможно, но можно его п..."
2,1608384240,"Data Science митап ВТБ, конкурс IT-проектов и ..."
3,1608365400,"Написать письмо коллеге, пройти собеседование ..."
4,1608301320,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
5,1608276780,🔥 Запустили бесплатный курс «Как стать SEO‑спе...
6,1608216360,Google Docs — это целый набор средств редактир...
7,1608191640,Приглашение на работу в Кремниевую долину — зв...
8,1608124503,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
9,1608103380,🏠 Запустили бесплатный онлайн-курс «Дизайн инт...


In [65]:
def date_convert (par):
    return datetime.utcfromtimestamp(par).strftime('%Y-%m-%d %H:%M:%S')
    
data['date_'] = data['date'].apply(date_convert)

del data['date'] 
data


,text,date_
0,Наш предновогодний digital-марафон продолжаетс...,2020-12-14 10:05:01
1,"Предсказать будущее невозможно, но можно его п...",2020-12-20 07:53:00
2,"Data Science митап ВТБ, конкурс IT-проектов и ...",2020-12-19 13:24:00
3,"Написать письмо коллеге, пройти собеседование ...",2020-12-19 08:10:00
4,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,...",2020-12-18 14:22:00
5,🔥 Запустили бесплатный курс «Как стать SEO‑спе...,2020-12-18 07:33:00
6,Google Docs — это целый набор средств редактир...,2020-12-17 14:46:00
7,Приглашение на работу в Кремниевую долину — зв...,2020-12-17 07:54:00
8,У нас сегодня Нето-викторина 🤩\n\nМы подготови...,2020-12-16 13:15:03
9,🏠 Запустили бесплатный онлайн-курс «Дизайн инт...,2020-12-16 07:23:00
